In [19]:
import plotly.express as px
import pandas as pd
from pathlib import Path
import numpy as np


In [113]:
import re
from collections import Counter

# open word features and plot the statistics of each text
word_features_folder = Path('stimuli/word_features/')

features = [
	'is_expert_technical_term',
	'is_general_technical_term',
	
]

topic = []
feature = []
values  = []
count = []

for wf_file in word_features_folder.glob('*.tsv'):
	text_id = wf_file.stem.split('_')[2]
	tsv = pd.read_csv(wf_file, sep='\t')
	
	if text_id.startswith('b'):
		for feat in features:

			c = Counter(tsv[feat].tolist())
			
			for k, v in c.items():
				topic.append('biology')
				feature.append(re.sub('_', ' ', feat).capitalize())
				values.append('is' if k == 1 else 'is not')
				count.append(v)
				
	
	else:
		for feat in features:

			c = Counter(tsv[feat].tolist())
			
			for k, v in c.items():
				topic.append('physics')
				feature.append(re.sub('_', ' ', feat).capitalize())
				values.append('is' if k == 1 else 'is not')
				count.append(v)

In [114]:
data_df = pd.DataFrame({
	'topic': topic,
	'feature': feature,
	'value': values,
	'count': count
})

In [115]:
sub_df = data_df.groupby(by=['topic', 'feature', 'value']).sum().reset_index()

# Create a bar plot using Plotly Express
# Create a bar plot using Plotly Express
fig = px.bar(
	sub_df, x='topic', y='count', color='value', title='Word feature counts per topic', barmode="stack", facet_col="feature", text_auto='s',
			labels={'count': 'Word count', 'feature': 'Feature', 'topic': '', 'value': ''} 
			 )
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig.update_traces(textposition="outside", cliponaxis=False)
